### Engine DB Queries for Wash Case Reports

In [1]:
import os; 
import pandas as pd; pd.set_option('mode.chained_assignment',None) 
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math
import re
import xlrd
from xlsxwriter.utility import xl_rowcol_to_cell
from random import sample
import time

from src.data_models.smartshelper import metrics, fixNum, unSMART, cparty, totalUP, wtf
from src.data_models.SmartsDataModel import SmartsDataModel
from src.utility.DataModelUtility import execute_query_data_frame

/Users/sqian/.pyenv/versions/3.7.0/envs/env_370/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


#### 1) dates & accounts of interest

In [2]:
alerting_1 = datetime.date(2019,8,1)
alerting_2 = datetime.date(2019,8,31)

worst = [22440, 811415]
remainingcpty = [1507866, 772544, 444259, 711292, 355845, 952523, 453659, 3032, 2647, 395052, 1595, 898579, 450782, 135552, 195483, 568127, 885724, 1403, 1169466]

pairlist = []

for i in worst: 
    for j in worst: 
        if i < j: pairlist.append([i,j])

#### 1A)  traded value($M) by Account (approximation- DO NOT USE)

In [3]:
query = """ 
        SELECT  created::date, account_id as pair, sum(price*quantity)/1000000 as TVmil, count(distinct event_id) as count
        from order_fill_event 
        where created between '{}' and '{}' and account_id in {}
        group by account_id, created::date
        order by created::date, account_id;
        """.format(alerting_1.strftime("%Y-%m-%d"),alerting_2.strftime("%Y-%m-%d"),str(tuple(worst)))

df = execute_query_data_frame(query,'engine')
output1a = df.groupby([df['created'].dt.month, 'pair']).sum().astype(int); output1a

INFO:root:Constructing Singleton
INFO:root:Using global configuration /Users/sqian/msv.yaml.
INFO:root:Completed loading configuration.
INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4p1)
INFO:paramiko.transport:Authentication (publickey) successful!


ssh_connect 0:00:00.858088


INFO:root:Loaded dataframe with 60 rows in 0:00:01.559712.


tvmil  count
created pair                
8       22440     330  52414
        811415    161  61616

#### 2) traded value($M) TOTAL EXCHANGE

In [4]:
query2 = """ 
        SELECT sum(firsttable.price * COALESCE(secondtable.open_price,1) * quantity/1000000) as valM
        
        from  
            (SELECT account_id, event_id, side, price, quantity, trading_pair, date_trunc('hour', created) start_time, substring(trading_pair::varchar,4,6) as trunc
            from order_fill_event 
            where created::date between '{t1}' and '{t2}' and side = 'buy') as firsttable 
        left join 
            (SELECT substring(trading_pair::varchar,1,3) as truncmatch, start_time, open_price
            from candles_1hr 
            where start_time::date between '{t1}' and '{t2}' and substring(trading_pair::varchar,4,6) = 'USD') as secondtable
        
        on firsttable.trunc = secondtable.truncmatch and firsttable.start_time = secondtable.start_time 
        
        
        """.format(t1 = alerting_1.strftime("%Y%m%d"), t2 = alerting_2.strftime("%Y%m%d"))

output2 = execute_query_data_frame(query2,'engine'); output2

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4p1)
INFO:paramiko.transport:Authentication (publickey) successful!


ssh_connect 0:00:00.563448


INFO:root:Loaded dataframe with 1 rows in 0:06:55.699061.


,valm
0,836.109636


In [5]:
query2 = """ 
        SELECT account_id, side, sum(firsttable.price * COALESCE(secondtable.open_price,1) * quantity/1000000) as valM
        
        from  
            (SELECT account_id, event_id, side, price, quantity, trading_pair, date_trunc('hour', created) start_time, substring(trading_pair::varchar,4,6) as trunc
            from order_fill_event 
            where created::date between '{t1}' and '{t2}' and account_id in {x}) as firsttable 
        left join 
            (SELECT substring(trading_pair::varchar,1,3) as truncmatch, start_time, open_price
            from candles_1hr 
            where start_time::date between '{t1}' and '{t2}' and substring(trading_pair::varchar,4,6) = 'USD') as secondtable
        
        on firsttable.trunc = secondtable.truncmatch and firsttable.start_time = secondtable.start_time 
        where account_id in {x}
        group by account_id, side
        """.format(t1 = alerting_1.strftime("%Y%m%d"), t2 = alerting_2.strftime("%Y%m%d"),x = str(tuple(remainingcpty)))

output2 = execute_query_data_frame(query2,'engine')
output2b = round(pd.pivot_table(output2,columns='side',index='account_id',values='valm'),2)

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4p1)
INFO:paramiko.transport:Authentication (publickey) successful!


ssh_connect 0:00:00.694764


INFO:root:Loaded dataframe with 38 rows in 0:04:09.674978.


In [6]:
output2b = round(pd.pivot_table(output2,columns='side',index='account_id',values='valm'),1)
output2b['TOTAL'] = output2b.sum(axis=1)
output2b['pct'] = round(output2b['TOTAL']/2325*100,0).astype(int).astype(str)+'%'
output2b.sort_values('TOTAL',ascending=False)

side,buy,sell,TOTAL,pct
account_id,,,,
711292,66.9,72.4,139.3,6%
1507866,49.0,50.1,99.1,4%
772544,46.3,46.1,92.4,4%
1595,40.2,39.5,79.7,3%
355845,31.7,35.0,66.7,3%
444259,21.9,25.1,47.0,2%
952523,11.8,11.8,23.6,1%
898579,6.4,3.8,10.2,0%
3032,5.4,3.2,8.6,0%


In [7]:
outputnew = output2b.sort_values('TOTAL',ascending=False)
nonzero = outputnew.loc[outputnew['pct'] != '0%']
zero = pd.DataFrame(outputnew.loc[outputnew['pct'] == '0%',['buy','sell','TOTAL']].sum(axis=0)).transpose()

In [8]:
outputnew2 = pd.concat([nonzero,zero],axis=0,sort=False).reset_index()
outputnew2.loc[outputnew2['index'] == 0,'pct'] = "<0%"

In [9]:
outputnew2.to_csv('~/Desktop/no.csv')

#### 3) traded value($M) by Account & Side

In [10]:
query3 = """ 
        SELECT account_id, trading_pair, firsttable.side, sum(firsttable.price * COALESCE(secondtable.open_price,1) * quantity/1000000) as valM
        
        from  
            (SELECT account_id, event_id, side, price, quantity, trading_pair, date_trunc('hour', created) start_time, substring(trading_pair::varchar,4,6) as trunc
            from order_fill_event 
            where created::date between '{t1}' and '{t2}' and account_id in {x}) as firsttable
        left join 
            (SELECT substring(trading_pair::varchar,1,3) as truncmatch, start_time, open_price
            from candles_1hr 
            where start_time::date between '{t1}' and '{t2}' and substring(trading_pair::varchar,4,6) = 'USD') as secondtable
        
        on firsttable.trunc = secondtable.truncmatch and firsttable.start_time = secondtable.start_time

        group by firsttable.side, account_id, trading_pair;
        """.format(t1 = alerting_1.strftime("%Y%m%d"), t2 = alerting_2.strftime("%Y%m%d"), x=str(tuple(worst)))

output3_D = execute_query_data_frame(query3,'engine')

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4p1)
INFO:paramiko.transport:Authentication (publickey) successful!


ssh_connect 0:00:00.580005


INFO:root:Loaded dataframe with 26 rows in 0:02:31.966052.


In [11]:
output3 = pd.pivot_table(output3_D, values='valm', index=['account_id','trading_pair'], columns=['side'], aggfunc=np.sum)
output3['TOTAL'] = output3.sum(axis=1)
round(output3,1)

side                       buy   sell  TOTAL
account_id trading_pair                     
22440      BCHBTC          0.3    0.2    0.5
           BCHUSD          0.9    1.1    2.0
           BTCUSD        131.5  166.9  298.4
           ETHBTC          1.1    0.8    1.9
           ETHUSD         14.1   13.4   27.5
           LTCUSD          2.2    1.2    3.4
           ZECUSD          0.1    0.1    0.2
811415     BCHUSD          1.2    1.2    2.5
           BTCUSD         67.5   67.8  135.2
           ETHBTC          1.6    1.5    3.1
           ETHUSD          6.9    6.9   13.8
           LTCUSD          5.5    5.4   10.9
           ZECUSD          0.2    0.2    0.4

In [12]:
output3.reset_index().groupby('account_id').sum()

side,buy,sell,TOTAL
account_id,,,
22440,150.09148,183.793566,333.885045
811415,82.83039,83.120548,165.950938


#### 4) traded value($M) by Account & Side

In [13]:
query4 = """ 
        SELECT account_id, sum(firsttable.price * COALESCE(secondtable.open_price,1) * quantity/1000000) as valM
        
        from  
            (SELECT account_id, event_id, side, price, quantity, trading_pair, date_trunc('hour', created) start_time, substring(trading_pair::varchar,4,6) as trunc
            from order_fill_event 
            where created::date between '{t1}' and '{t2}' and account_id in {x}) as firsttable
        left join 
            (SELECT substring(trading_pair::varchar,1,3) as truncmatch, start_time, open_price
            from candles_1hr 
            where start_time::date between '{t1}' and '{t2}' and substring(trading_pair::varchar,4,6) = 'USD') as secondtable
        
        on firsttable.trunc = secondtable.truncmatch and firsttable.start_time = secondtable.start_time
        
        where account_id = {y} and event_id in 
            (SELECT distinct event_id from order_fill_event
            where created between '{t1}' and '{t2}' and account_id = {z}
            group by event_id) 
        
        group by account_id;
        """

output4 = pd.DataFrame()

for i in pairlist:
    temp = execute_query_data_frame(query4.format(t1 = alerting_1.strftime("%Y%m%d"), t2 = alerting_2.strftime("%Y%m%d"), x=str(tuple(worst)), y = i[0], z = i[1]),'engine')
    temp['pair'] = str(i)
    output4 = output4.append(temp, sort=True)
    
output4

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4p1)
INFO:paramiko.transport:Authentication (publickey) successful!


ssh_connect 0:00:00.691275


INFO:root:Loaded dataframe with 1 rows in 0:00:01.626471.


,account_id,pair,valm
0,22440,"[22440, 811415]",47.974115
